<a href="https://colab.research.google.com/github/restrepo/BSM-Submodules/blob/master/SARAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SARAH configuration files generator

In [1]:
import os
if os.getcwd()=='/content':
    !wget -O SARAH.py https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/SARAH.py 2>/dev/null
    !wget -O cmdlike.py https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/cmdlike.py 2>/dev/null
    !git clone https://github.com/restrepo/SARAH.git 2>/dev/null
    !mkdir -p JSON
    os.chdir('JSON')
    !wget -O fullparticles.json https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/JSON/fullparticles.json 2> /dev/null
    !wget -O fullparticlesnames.json https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/JSON/fullparticlesnames.json 2> /dev/null
    !wget -O fullparametersnames.json https://raw.githubusercontent.com/restrepo/BSM-Submodules/master/JSON/fullparametersnames.json 2> /dev/null
    os.chdir('../')

In [2]:
import pandas as pd
import json
import re
import numpy as np
import sys
from SARAH import *
pd.set_option('display.max_colwidth',200)

## Read `MODEL.m` file into an standarized list of dictionaries 

### Design
#### Class inheritance
* Basic object
```
list_of_list_of_dictionaries -> Particles -> Parameters
```
* Methods
```
SARAH -> update_SARAH_particles -> update_SARAH_parameters -> update_SARAH_SPheno -> update_SARAH
```


#### USAGE:
1. Load a `SARAH` Model File
2. Warns if a `particle` or `parameter` is not yet defined
3. Build missing `particles` or `parameters` and update predefined particles loaded in `__init__`
4. Export `SARAH` auxiliarly files:
   * `particles.m`   
   * `parameters.m`
   * `SPheno.m`

In [3]:
from SARAH import *

In [4]:
s=SARAH(model='SSDM')
#newp=s.parse_model_particles()

In [5]:
s.parse_model_particles()

[{'Definition': 'GaugeES',
  'Field': 'VB',
  'Parents': None,
  'Properties': {'Coupling': ' g1',
   'Group': '   U[1]',
   'Index': ' hypercharge',
   'Lorentz': 'Vector',
   'SSB': ' g1'}},
 {'Definition': 'GaugeES',
  'Field': 'VWB',
  'Parents': None,
  'Properties': {'Coupling': '        g2',
   'Group': ' SU[2]',
   'Index': ' left',
   'Lorentz': 'Vector',
   'SSB': '        g2'}},
 {'Definition': 'GaugeES',
  'Field': 'VG',
  'Parents': None,
  'Properties': {'Coupling': '       g3',
   'Group': '  SU[3]',
   'Index': ' color',
   'Lorentz': 'Vector',
   'SSB': '       g3'}},
 {'Definition': 'WeylFermionAndIndermediate',
  'Field': 'q',
  'Parents': None,
  'Properties': {'Groups': ['1/6', '2', '3', '1'],
   'Lorentz': 'WeylFermion',
   'NF': '3',
   'multiplet': ['uL', 'dL']}},
 {'Definition': 'WeylFermionAndIndermediate',
  'Field': 'l',
  'Parents': None,
  'Properties': {'Groups': ['-1/2', '2', '1', '1'],
   'Lorentz': 'WeylFermion',
   'NF': '3',
   'multiplet': ['vL', 'e

In [6]:
pd.DataFrame(s.modelparticles)

,Block,Definition,Field,Parents,Properties,rotation
0,NaN,GaugeES,VB,None,"{'Lorentz': 'Vector', 'Coupling': ' g1', 'SSB': ' g1', 'Index': ' hypercharge', 'Group': ' U[1]'}",NaN
1,NaN,GaugeES,VWB,None,"{'Lorentz': 'Vector', 'Coupling': ' g2', 'SSB': ' g2', 'Index': ' left', 'Group': ' SU[2]'}",NaN
2,NaN,GaugeES,VG,None,"{'Lorentz': 'Vector', 'Coupling': ' g3', 'SSB': ' g3', 'Index': ' color', 'Group': ' SU[3]'}",NaN
3,NaN,WeylFermionAndIndermediate,q,None,"{'Groups': ['1/6', '2', '3', '1'], 'Lorentz': 'WeylFermion', 'multiplet': ['uL', 'dL'], 'NF': '3'}",NaN
4,NaN,WeylFermionAndIndermediate,l,None,"{'Groups': ['-1/2', '2', '1', '1'], 'Lorentz': 'WeylFermion', 'multiplet': ['vL', 'eL'], 'NF': '3'}",NaN
5,NaN,WeylFermionAndIndermediate,d,None,"{'Groups': ['1/3', '1', '-3', '1'], 'Lorentz': 'WeylFermion', 'NF': '3'}",NaN
6,NaN,WeylFermionAndIndermediate,u,None,"{'Groups': ['-2/3', '1', '-3', '1'], 'Lorentz': 'WeylFermion', 'NF': '3'}",NaN
7,NaN,WeylFermionAndIndermediate,e,None,"{'Groups': ['1', '1', '1', '1'], 'Lorentz': 'WeylFermion', 'NF': '3'}",NaN
8,NaN,WeylFermionAndIndermediate,H,None,"{'Groups': ['1/2', '2', '1', '1'], 'Lorentz': 'Scalar', 'multiplet': ['Hp', 'H0'], 'NF': '1'}",NaN
9,NaN,WeylFermionAndIndermediate,S,None,"{'Groups': ['0', '1', '1', '-1'], 'Lorentz': 'Scalar', 'Real': True, 'NF': '1'}",NaN


## Update particles

### How to build a function for the `self.update_particles` method
For one Specific Type of Particles: `self.update_particles(f,Definition,Lorentz,args)` is expected to filter the set of particles
by using the  `Definition`  and `Lorentz` arguments, and pass the resulting list of particles (as an object `Particles`) to function `f(particle,*args)`, where `particle` is each one of the elements of the filtered list of particles.  The function must return another `Particles` object with the help of the extra arguments passed through the optional tuple: `args=(...)`

The main purpose of the generic function `f` is either to 
1. add additional info to an existing particle: Define a function with a mandatory `particle` argument which returns the  same particle with additional info like `p['Description]`. The recommended name for the function in such a case is `get_Specific_Type_of_Particles_Descriptions(particle,...)`
1. generate a new non-existing particle. In such a case the recommended name for the function is `get_Specific_Type_of_Particles(particle,...)`. Each existing particle  must generare one and only one new particle

__Example__:

Define
```python
def get_WeylFermion_Description(p,particles):
    p.get('Properties')['update_Description']={'LaTeX' : get_weylfermion_LaTeX( p.get('Field') )}
    return p
```
and use
```python
self.update_particles( get_WeylFermion_Descriptions, 
                      Definition='WeylFermionAndIndermediate',
                      Lorentz='WeylFermion')
```

In [7]:
def get_4Spinor_Descriptions(particle,particles):
    """
    Function which that take the dictionary particle and adds: 
    'Description' or 'Properties' -> 'update_Description'
    To the particle with 'Field' some 4Spinor
    """
    p=particle
    gprt=particles.loc[ p.get('Parents') ]
    if gprt[0].get('rotation'):
        gprt=particles.loc[gprt[0].get('Parents')]
    cmpt=gprt[0].get('Field')
    gprt=particles.loc[gprt[0].get('Parents')]
    if get_abs_groups( gprt[0].get('Properties').get('Groups'), u1_abs='1/6',su2='2',su3_abs='3' ):
        if cmpt==gprt[0].get('Properties').get('multiplet')[0]:
            p['Description']='Up-Quarks'
        else:
            p['Description']='Down-Quarks'
    if get_abs_groups( gprt[0].get('Properties').get('Groups'), u1_abs='1/2',su2='2',su3_abs='1' ):
        if cmpt==gprt[0].get('Properties').get('multiplet')[0]:
            p['Description']='Neutrinos'
        else:
            p['Description']='Leptons'
    return p

def get_WeylFermion_Descriptions(p):
    '''
    All the Weyl Fermions are used. To filter simplified iso-singlet names, use:
        if not p.get('Parents') and not p.get('Properties').get('multiplet'):
            RETURN=False
        #Empty dic: {}, must be returned when RETURN is False
    '''
    p.get('Properties')['update_Description']={'LaTeX' : get_weylfermion_LaTeX( p.get('Field') )}
    return p

def get_gauge_vectors_Descriptions(p,particles,
                di={'U[1]' :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost' },
                    'SU[2]':{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                    'SU[3]':{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}):
    gr=p.get('Properties').get('Group').strip()
    if di.get( gr  ):
        p['Description']=di.get(gr).get('Description')
    return p

def get_ghosts(p,particles,
                di={'U[1]' :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost' },
                    'SU[2]':{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                    'SU[3]':{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}):
    gr=p.get('Properties').get('Group').strip()
    if di.get( gr  ):
        p=particles.apply_filter(lambda d: d.get('Description')==di.get(gr).get('Ghost'))[0]
        return p
    else:
        return {}
    
def get_boson_vectors_Descriptions(p,particles,partialparticles,
                      di={'B-Boson':'Photon',
                          'W-Bosons':{1:'W+ - Boson',3:'Z-Boson'}},
                      goldston={
                          'W-Bosons':{1:'Charged Higgs',3:'Pseudo-Scalar Higgs'}
                                      }):
    '''
    Assumptions:
    1) The Goldston bosons are the SM ones and this function
    must be called after the scalar definitions
    '''
    pP=p.get('Parents')
    if not pP:
        pP=''
    abelian=pP.split('[')
    BV=particles.loc[abelian[0]]
    try:
        dscr=BV[0].get('Description')
    except IndexError:
        dscr=''
        
    la=len(abelian)
    gltn=Particles([])
    if la==1:
        get_dscrp=di.get(dscr)
    else:
        try:
            i=eval( re.search('\[\s*([0-9]+)\s*\]',pP).groups()[0] )
        except (IndexError,AttributeError):
            i=-1
            
        try:
            get_dscrp=di.get(dscr).get(i)
            gltn=partialparticles.apply_filter(
                lambda d: d.get('Description')==goldston.get(dscr).get(i) 
                     )
        except AttributeError:
            get_dscrp=''
            gltn=Particles([])
            
    if get_dscrp:
        p['Description']=get_dscrp
        
    if gltn.size()==1:
        p['Properties']['update_Description']={'Goldstone':gltn.get('Field')[0]}        
        

    return p

def get_boson_vectors_ghosts(p,particles,
                      di={'B-Boson':'Photon Ghost',
                          'W-Bosons':{1:'Positive W+ - Boson Ghost',
                                      3:'Z-Boson Ghost'}}):
    np={}
    pP=p.get('Parents')
    abelian=pP.split('[')
    BV=particles.loc[abelian[0]]
    dscr=BV[0].get('Description')
    la=len(abelian)
    gltn=Particles([])
    if la==1:
        get_dscrp=di.get(dscr)
    else:
        try:
            i=eval( re.search('\[\s*([0-9]+)\s*\]',pP).groups()[0] )
        except (IndexError,AttributeError):
            i=-1
            
        get_dscrp=di.get(dscr).get(i)

    if get_dscrp:
        np['Field']      =re.sub('^V','g',  p.get('Field'))
        np['Description']=get_dscrp
        np['Block']      = 'GaugeSector'
        np['Definition'] = 'EWSB'
        np['Properties'] ={'Lorentz': 'Ghost'}
        return np
    else:
        return {}

def get_IntermediateScalars_Definitions(p):
    if get_abs_groups( 
         p.get('Properties').get('Groups'),u1_abs='1/2',su2='2',su3_abs='1'):
        if p.get('Properties').get('Groups')[0].find('-')==-1:
            p.get('Properties')['charge']=['Charged','Neutral']
        else:
            p.get('Properties')['charge']=['Neutral','Charged']

    p.get('Properties')['update_Description']={
                                               'PDG'       : [0],
                                               'Width'     : 0,
                                               'Mass'      : 'Automatic',
                                               'LaTeX'     : p.get('Field'),
                                               'OutputName': p.get('Field') }
    return p


def get_GaugeEScalars_Definitions(p,H,particles,
                                      pids={'PDG':[6666635],
                                           'PDG.IX':[101000002],
                                           'FeynArtsNr':[10]
                                           }):
    if H.size()==1 and p.get('Parents')==H.get('Field')[0]:
        #Standard model Higgs
        p.get('Properties')['update_Description']={'PDG'  :[0],
                                               'Width':0,
                                               'Mass' : 'Automatic',
                                               'OutputName' : p.get('Field')
                                               }

        l=H[0].get('Properties').get('multiplet')
        Hp0=l.index(p.get('Field'))
        if H[0].get('Properties').get('charge')[Hp0]=='Charged':
            p.get('Properties')['ElectricCharge']=1
            p.get('Properties')['update_Description']['FeynArtsNr']=2
            p.get('Properties')['update_Description']['LaTeX']='H^+'
        elif H[0].get('Properties').get('charge')[Hp0]=='Neutral':
            p.get('Properties')['ElectricCharge']=0
            p.get('Properties')['update_Description']['FeynArtsNr']=1
            p.get('Properties')['update_Description']['LaTeX']='H^0'
    #INERT scalars
    kk=particles.loc[p.get('Parents')]
    #singlet real scalar
    if get_abs_groups( kk.get('Properties').get('Groups')[0],u1_abs='0',su2='1',su3_abs='1'):
        p['Definition']='EWSB'
        p['Description']='Singlet Inert Scalar'
        #TODO: Get from fullparticlesnames.json
        p['update_Description']= {'PDG' : [pids.get('PDG')[0]],
                                  'PDG.IX' : [pids.get('PDG.IX')[0]],
                                  'FeynArtsNr' : pids.get('FeynArtsNr')[0],
                                  'Mass': 'LesHouches',               
                                  'LaTeX' : 'S',
                                  'ElectricCharge' : 0,
                                  'LHPC' : ['gold'],
                                  'OutputName' : 'Ss'
                                 }
    return p

def get_EWSBScalars_Definitions(p,particles):
    if p.get('Properties').get('CP')=='Real':
        pp=particles.apply_filter(lambda d: d.get('Description')=='Higgs')
        try:
            pphh=pp[0]
        except IndexError:
            pphh={}
        if pphh:
            p['Description']=pphh.get('Description')
            p.get('Properties')['update_Description']=pphh.get('Properties').get(
                                                            'update_Description')
    elif p.get('Properties').get('CP')=='Imaginary':
        pp=particles.apply_filter(lambda d: d.get('Description')==
                                                        'Pseudo-Scalar Higgs')
        try:
            ppA0=pp[0]
        except IndexError:
            ppA0={}
        if ppA0:
            p['Description']=ppA0.get('Description')
            p.get('Properties')['update_Description']=ppA0.get('Properties').get(
                                                            'update_Description')
        
    return p    

def get_EWSBScalars(p,H,particles):
    '''
    Only Works if 
    H=self.update_particles(get_IntermediateScalars_Definitions,
                               Definition='WeylFermionAndIndermediate',
                               Lorentz='Scalar').size()==1
    '''
    pp={}
    if H.size()==1 and p.get('Field')==H.get('Field')[0]:
        kk=particles.apply_filter(lambda d: d.get('Name')=='Charged Higgs')
        pp=kk[0]
        Hp=H[0].get('Properties').get('charge').index('Charged')
        try:
            Hpname=H[0].get('Properties').get('multiplet')[Hp]
            pp['Field']  = Hpname
            pp['Parents']= Hpname
        except IndexError:
            pass
    return pp

def fix_gluon(d):
    if d.get('Description')=='Gluon':
        d['Definition']='EWSB'
    if d.get('Description')=='Gluon Ghost':
        d['Definition']='EWSB'
    return d

In [8]:
class update_SARAH_particles(SARAH):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)

    def update_fermions(self):
        NP=Particles([])
        NP=self.update_particles(get_4Spinor_Descriptions,
                                Definition='EWSB',
                                Lorentz='DiracSpinor',
                                args=(self.modelparticles))
        NP=NP+self.update_particles(get_4Spinor_Descriptions,
              Definition='EWSB',Lorentz='MajoranaSpinor',args=(self.modelparticles))
        NP=NP+self.update_particles( get_WeylFermion_Descriptions, 
                                Definition='WeylFermionAndIndermediate',
                                Lorentz='WeylFermion')
        return NP

    def update_gauge_bosons(self):
        NP=Particles([])
        NP=NP+self.update_particles(get_gauge_vectors_Descriptions,
                                Definition='GaugeES',
                                Lorentz='Vector',
                                args=(self.modelparticles,
                            {'U[1]'  :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost'},
                             'SU[2]' :{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                             'SU[3]' :{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}))
        NP=NP+self.update_particles(get_ghosts,
                                Definition='GaugeES',
                                Lorentz='Vector',
                                args=(self.particles,
                            {'U[1]'  :{'Description':'B-Boson' ,'Ghost':'B-Boson Ghost'},
                             'SU[2]' :{'Description':'W-Bosons','Ghost':'W-Boson Ghost'},
                             'SU[3]' :{'Description':'Gluon'   ,'Ghost':'Gluon Ghost'}}))
        NP=NP+self.update_particles(get_boson_vectors_Descriptions,
                                Definition='EWSB',
                                Lorentz='Vector',
                                args=(NP,
                                      self.updated_modelparticles,
                            {'B-Boson':'Photon',
                             'W-Bosons':{1:'W+ - Boson',3:'Z-Boson'}}))
        NP=NP+self.update_particles(get_boson_vectors_ghosts,
                                Definition='EWSB',
                                Lorentz='Vector',
                                args=(NP,
                            {'B-Boson':'Photon Ghost',
                             'W-Bosons':{1:'Positive W+ - Boson Ghost',
                                         3:'Z-Boson Ghost'}}))
        #Special case: Negative ghosts
        negative_ghost='Negative W+ - Boson Ghost'
        NP=NP+self.update_particles(get_boson_vectors_ghosts,
                                Definition='EWSB',
                                Lorentz='Vector',
                                args=(NP,
                            {'B-Boson':'',
                             'W-Bosons':{1:negative_ghost,
                                         3:''}})).apply_filter(
                            lambda d: d.get('Description')==negative_ghost
                                                      ).apply(fC)
        
        #Special case: Fix Gluons
        NP=Particles(NP.apply(fix_gluon))
        
        return NP

    def update_scalars(self):
        '''
        The order matters!
        '''
        NP=Particles([])
        
        H=self.update_particles(get_IntermediateScalars_Definitions,
                                    Definition='WeylFermionAndIndermediate',
                                    Lorentz='Scalar')
        NP=NP+H
        
        SMVEV=False
        VEVs=self.modelparticles.apply_filter(
            lambda d: d.get('Block')=='VEVs' )
        if VEVs.size()==2:
            SMVEV=True
        
        #SM-like 'WeylFermionAndIndermediate' (WFI) states, `HSM` + `INERTs` WFI states
        if SMVEV:
            HSM_name=self.modelparticles.loc[VEVs.get('Parents')[0]].get('Parents')[0]    
            HSM=H.loc[HSM_name]
            INERTs=H.apply_filter( lambda d: d.get('Field')!=HSM_name)
        #TODO: Multi-vev potentials
        else:
            HSM=H
            INERTs=Particles([])        

        #Process EWSB standard model particles    
        NP=NP+self.update_particles(get_EWSBScalars_Definitions,
                                    Definition='EWSB',
                                    Lorentz='Scalar',args=(self.particles))
        
            
        #For SM-like Fields Descriptions added,
        #WARNING: Inert Scalars 
        HpH0=self.update_particles(get_GaugeEScalars_Definitions,Definition='GaugeES',
                                    Lorentz='Scalar',
                                    args=(HSM,
                                          self.modelparticles,
                                          {'PDG':[6666635],
                                           'PDG.IX':[101000002],
                                           'FeynArtsNr':[10]
                                          }
                                     ))
        NP=NP+HpH0

        #Repeat 'WeylFermionAndIndermediate' Hp goldstone-field as EWSB field
        # to accomodate weird SARAH design
        kk=self.update_particles(get_EWSBScalars,
                                    Definition='WeylFermionAndIndermediate',                    
                                    Lorentz='Scalar',args=(HSM,self.particles))
        kk=kk.apply_filter(lambda d: isinstance(d.get('Field'),str))

        NP=NP+kk
        
        #non 'WeylFermionAndIndermediate' Beyond  SM scalar field

        return NP
            
    def update_modelparticles(self):            
        #must be first
        self.updated_modelparticles=self.update_scalars()

        # GaugeBosons
        self.updated_modelparticles=self.updated_modelparticles+self.update_gauge_bosons()

        #Scalars
        self.updated_modelparticles=self.updated_modelparticles+self.update_fermions()

        return self.updated_modelparticles

In [9]:
us=update_SARAH_particles(model='SSDM')
#newp=us.parse_model_particles()
kk=us.update_modelparticles()
self=us

## SM like scalar potential plus inert  scalars

In [10]:
self.updated_modelparticles.apply_filter(lambda d:
                d.get('Properties').get('Lorentz')=='Scalar')

[{'Definition': 'WeylFermionAndIndermediate',
  'Field': 'H',
  'Parents': None,
  'Properties': {'Groups': ['1/2', '2', '1', '1'],
   'Lorentz': 'Scalar',
   'NF': '1',
   'charge': ['Charged', 'Neutral'],
   'multiplet': ['Hp', 'H0'],
   'update_Description': {'LaTeX': 'H',
    'Mass': 'Automatic',
    'OutputName': 'H',
    'PDG': [0],
    'Width': 0}}},
 {'Definition': 'WeylFermionAndIndermediate',
  'Field': 'S',
  'Parents': None,
  'Properties': {'Groups': ['0', '1', '1', '-1'],
   'Lorentz': 'Scalar',
   'NF': '1',
   'Real': True,
   'update_Description': {'LaTeX': 'S',
    'Mass': 'Automatic',
    'OutputName': 'S',
    'PDG': [0],
    'Width': 0}}},
 {'Block': 'VEVs',
  'Definition': 'EWSB',
  'Description': 'Higgs',
  'Field': 'hh',
  'Parents': 'H0',
  'Properties': {'CP': 'Real',
   'Coefficient': '1/Sqrt[2]',
   'Lorentz': 'Scalar',
   'update_Description': {'PDG': [25], 'PDG.IX': [101000001]},
   'vev': 'v'}},
 {'Block': 'VEVs',
  'Definition': 'EWSB',
  'Description': 

## Parse and update parameters

In [11]:
class update_SARAH_parameters(update_SARAH_particles):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def Lagrangian_Couplings_Descriptions(self):
        '''
        '''
        dd=self.Lagrangian
        smdict=self.Lagrangian_Couplings
        particles=self.updated_modelparticles

        for k in dd.keys():
            if (sorted_equality( smdict['Up-Yukawa-Coupling']['Lorentz'],dd[k]['Lorentz'] ) and
                sorted_equality( smdict['Up-Yukawa-Coupling']['hypercharge'],dd[k]['hypercharge'] ) ):
                #Get Higgs from scalar part
                for i in range(len(dd[k]['Lorentz'])):
                    if dd[k]['Lorentz'][i]=='Scalar':
                        H=dd[k]['fields'][i]
                        smdict['Up-Yukawa-Coupling']['Coupling']=k
                        smdict['Up-Yukawa-Coupling']['Higgs']=H
                        smdict['Up-Yukawa-Coupling']['fields']=dd[k]['fields']
                #Use obtained Higgs to obtain diagonal form
                if not H:
                    sys.exit('Higgs doublet field symbol not found!')
                for i in range(len(dd[k]['Lorentz'])):                
                    if dd[k]['Lorentz'][i]=='WeylFermion':
                        mltp=get_multiplet(dd[k]['fields'][i],particles)
                        for p in mltp.keys():
                            if mltp[p].get('pos')=='Up':
                                vev=get_higgs_vev(H,particles)
                                smdict['Up-Yukawa-Coupling'
                                      ]['update_Description'
                                      ]['DependenceNum']=get_diagonal_basis(
                                                         vev,p,particles)                

        if not smdict.get('Up-Yukawa-Coupling'):
            sys.exit('"Up-Yukawa-Coupling" NOT FOUND!' )
        lk=list(dd.keys())
        try:
            lk.remove( smdict['Up-Yukawa-Coupling']['Coupling'] )
        except KeyError:
            pass

        lds=list(smdict.keys())
        lds.remove('Up-Yukawa-Coupling')
        for ds in lds:
            for k in lk:
                if (sorted_equality( smdict[ds]['Lorentz'],dd[k]['Lorentz'] ) and
                    sorted_equality( smdict[ds]['hypercharge'],dd[k]['hypercharge'] ) ):
                    smdict[ds]['Coupling']=k
                    smdict[ds]['fields']=dd[k]['fields']
                    if H:
                        smdict[ds]['Higgs']=H
                    for i in range(len(dd[k]['Lorentz'])):
                        if dd[k]['Lorentz'][i]=='WeylFermion':
                            mltp=get_multiplet(dd[k]['fields'][i],particles)
                            for p in mltp.keys():
                                if mltp[p].get('pos')=='Down':
                                    Hk=[dd[k].get('fields')[i] for i in range(len( dd[k]['Lorentz'] ))
                                         if dd[k]['Lorentz'][i]=='Scalar' ][0]
                                    vev=get_higgs_vev(Hk,particles)
                                    smdict[ds]['update_Description'
                                          ]['DependenceNum']=get_diagonal_basis(
                                                             vev,p,particles)
        self.Lagrangian_Couplings=smdict
        return smdict
            
    def get_rotations(self):
        rotation={}
        smdict=self.Lagrangian_Couplings
        particles=self.updated_modelparticles
        for k in smdict.keys():
            try:
                fields=range(len(smdict[k].get('fields')))
            except TypeError:
                fields=[]
            for i in fields:
                if smdict[k]['Lorentz'][i]=='WeylFermion':
                    mltp=particles.apply_filter(lambda d: d.get('Parents')==smdict[k]['fields'][i])
                    if mltp.size()==2:
                        chiral='Left'
                    elif mltp.size()==1:
                        chiral='Right'
                    else:
                        chiral=None

                    j=0
                    for p in mltp.get('Field'):
                        j=j+1
                        prt=particles.loc[p]
                        rot=particles.apply_filter(
                            lambda d: str(d.get('Parents')).find(p)>-1
                                                 ).get('rotation')[0]
                        if isinstance(rot,str):
                            rotation[p]={rot:{}}
                            dscr='Mixing-Matrix'
                            if chiral=='Left':
                                if j==1:
                                    dscr='{}-Up-{}'.format(chiral,dscr)
                                elif j==2:
                                    if re.search('^[dD]',p):
                                        dscr='{}-Down-{}'.format(chiral,dscr)
                                    elif re.search('^[eE]',p):
                                        dscr='{}-Lepton-{}'.format(chiral,dscr)


                            elif chiral=='Right':
                                if re.search('^[uU]',p):
                                    dscr='{}-Up-{}'.format(chiral,dscr)
                                elif re.search('^[dD]',p):
                                    dscr='{}-Down-{}'.format(chiral,dscr)
                                elif re.search('^[eE]',p):
                                    dscr='{}-Lepton-{}'.format(chiral,dscr)
                                else:
                                    dscr=None
                            rotation[p][rot]['Description']=dscr

        self.rotations=rotation
        return rotation
    
    def get_couplings(self):
        particles=self.updated_modelparticles
        rotation=self.rotations

        prtng=particles.apply_filter(
                lambda d: str(d.get('Description')).lower().find('ghost')==-1
                     )
        grps=prtng.apply_filter(
                lambda d: d.get('Properties').get('Group')!=None
                     ).apply(
                lambda d: d.get('Properties').get('Group')
                )

        G={}
        coupling={}
        for g in grps:
            G=prtng.apply_filter(lambda d: d.get('Properties').get('Group')==g)
            V=G[0].get('Field')
            try:
                VV=prtng.apply_filter(lambda d: str(d.get('Parents')).find(V)>-1)
                # Non-Abelian case with SSB
                if VV.size()==2:
                    key='Properties'
                    pattern='conj\[\w+\]'
                    VV=VV.apply_filter( lambda d: re.search( pattern, str(d.get(key))  ) )

                f=VV[0]['Field']
                r=VV[0]['rotation']
            except IndexError:
                f=''  
            c=G[0].get('Properties').get('Coupling').strip()
            if g.find('U[1]')>-1:
                if f:
                    rotation[f]={r: {'Description':"Photon-Z Mixing Matrix"}}
                coupling[c]={'Description':'Hypercharge-Coupling'}
            if g.find('SU[2]')>-1:
                if f:
                    rotation[f]={r: {'Description':"W Mixing Matrix",
                                      'update_Description':
                                            {'Dependence' :r'''1/Sqrt[2] {{1, 1},
                                      {\[ImaginaryI],-\[ImaginaryI]}}''' }}}
                coupling[c]={'Description':'Left-Coupling'}
            if g.find('SU[3]')>-1:
                coupling[c]={'Description':'Strong-Coupling'}
        self.couplings=coupling
        return coupling
    
    def get_constants(self):
        self.constants={}
        self.constants['AlphaS']= { 'Description'  : 'Alpha Strong'}
        self.constants['e']     = { 'Description'  :  'electric charge'} 
        self.constants['Gf']    = { 'Description'  :  "Fermi's constant"}
        self.constants['aEWinv']= { 'Description'  :  'inverse weak coupling constant at mZ'}
        self.constants['mH2']   = { 'Description'  :  'SM Higgs Mass Parameter'}
        return self.constants
    
    def update_Lagrangian(self):
        particles=self.updated_modelparticles
        rotation=self.rotations
        coupling=self.couplings
        H=self.Lagrangian_Couplings['SM Higgs Selfcouplings'
                                   ].get('Higgs')
        if H:
            H0=get_H0(H,particles)
            if H0.size()>0:
                hh=get_hh(H0,particles).get('Field')[0]
                vev=get_hh(H0,particles).get('Properties')[0].get('vev')
                if hh and vev:
                    self.Lagrangian_Couplings['SM Higgs Selfcouplings'
                          ]['update_Description']={
                            'DependenceNum': 
                        r'Mass[{}]^2/({}^2)'.format(hh,vev) }

        self.Lagrangian_Couplings['EW-VEV']={}
        self.Lagrangian_Couplings['EW-VEV']['Coupling']=vev
        self.Lagrangian_Couplings['EW-VEV']['update_Description']={}
        self.Lagrangian_Couplings['EW-VEV']['update_Description']['DependenceSPheno']=None
        self.Lagrangian_Couplings['EW-VEV']['update_Description']['OutputName']='vvSM'
        VWp=''
        g2=''
        for k in rotation.keys():
            if list( rotation[k].values() )[0].get('Description')=='W Mixing Matrix':
                VWp=k
        for c in coupling.keys():
            if coupling[c].get('Description')=='Left-Coupling':
                g2=c
        self.Lagrangian_Couplings['EW-VEV'
                                 ]['update_Description'
                                  ]['DependenceNum']=r'Sqrt[4*Mass[{}]^2/({}^2)]'.format(VWp,g2)

        for k in rotation:
            if list( rotation[k].values() )[0].get('Description')=='Photon-Z Mixing Matrix':
                VP=k
                ZZ=list(rotation[k].keys())[0]
        fz=particles.apply_filter( lambda d: d.get('rotation')==ZZ )
        VZ=fz.apply_filter(lambda d: d.get('Field')!=VP)[0]['Field']
        self.Lagrangian_Couplings['Weinberg-Angle']={}
        self.Lagrangian_Couplings['Weinberg-Angle']['Coupling']='ThetaW'
        self.Lagrangian_Couplings['Weinberg-Angle']['update_Description']={}
        self.Lagrangian_Couplings['Weinberg-Angle']['update_Description'
             ]['DependenceNum']='ArcSin[Sqrt[1 - Mass[{}]^2/Mass[{}]^2]]'.format(
                                                                   VWp,VZ )
        
    def update_modelparameters(self):
        smdict=self.Lagrangian_Couplings
        rotation=self.rotations
        coupling=self.couplings
        constants=self.constants
        parameters=[]

        for k in smdict:
            d={}
            d['Description']=k
            d['Name']=d['Description']
            d['Properties']=smdict[k]
            d['Symbol']=d.get('Properties').get('Coupling')
            d['Class']='Lagrangian'
            kk=parameters.append(d)
            
        #rotation will be altered
        rt=copy.deepcopy(rotation)
        for k in rt.keys():
            for r in rt[k].keys():
                d={}
                d['Symbol']=r
                d['Description']=rt[k][r].get('Description')
                d['Name']=d['Description']
                d['Class']='Rotation'
                kk=rt[k][r].pop('Description')
                if rt[k][r]:
                    d['Properties']=rt[k][r]
                else:
                    d['Properties']={}
                parameters.append(d)
        del(rt)
        
        for k in coupling.keys():
            d={}
            d['Symbol']=k
            d['Description']=coupling[k].get('Description')
            d['Name']=d['Description']
            d['Class']='Coupling'
            d['Properties']={}
            parameters.append(d)
            
        for k in constants.keys():
            d={}
            d['Symbol']=k
            d['Description']=constants[k].get('Description')
            d['Name']=d['Description']
            d['Class']='Constant'
            d['Properties']={}
            parameters.append(d)
            
        parameters=Parameters(parameters,index='Symbol')
        self.updated_modelparameters=parameters
        return parameters        

    # Parse parameters
    def parse_model_parameters(self):
        #2)
        kk=self.update_modelparticles()
        kk=self.parse_Lagrangian()
        kk=self.Lagrangian_Couplings_Descriptions()
        kk=self.get_rotations()
        kk=self.get_couplings()
        kk=self.get_constants()
        kk=self.update_Lagrangian()
        kk=self.update_modelparameters()
        return kk

In [12]:
us=update_SARAH_parameters(model='SSDM')
#kk=us.parse_model_particles()
#kk=us.update_modelparticles()
kk=us.parse_model_parameters()

In [13]:
pd.DataFrame(us.updated_modelparameters )[:1]

,Class,Description,Name,Properties,Symbol
0,Lagrangian,SM Higgs Selfcouplings,SM Higgs Selfcouplings,"{'Higgs': 'H', 'Coupling': 'Lambda1/2', 'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'], 'update_Description': {'DependenceNum': 'Mass[hh]^2/(v^2)'}, 'hypercharge': ['1/2', '1/2', '1/2', '1/2'...",Lambda1/2


## SPheno

In [15]:
class update_SARAH(update_SARAH_parameters):
    def __init__(self,*args, **kwargs):
        self.SPheno={'Properties':{
         'OnlyLowEnergySPheno' : True,
         'AddTreeLevelUnitarityLimits':True,
        'MINPAR':[],
        'ParametersToSolveTadpoles':[],
        'BoundaryLowScaleInput':[],
        'ListDecayParticles':[],
        'ListDecayParticles3B': [],
        'RenConditionsDecays':[
               ['dCosTW', '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
               ['dSinTW', '-dCosTW/Tan[ThetaW]'],
               ['dg2', '1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiVZVP*Tan[ThetaW])/MVZ^2)'],
               ['dg1', 'dg2*Tan[ThetaW]+g2*dSinTW/Cos[ThetaW]- dCosTW*g2*Tan[ThetaW]/Cos[ThetaW]']
             ],
        'DEFINITION': {'MatchingConditions':
                 []},
        'DefaultInputValues' :{}
           },
        'Index':{'OnlyLowEnergySPheno':0,'MINPAR':1, 'ParametersToSolveTadpoles':2, 
                'BoundaryLowScaleInput':3, 'DEFINITION':4, 'ListDecayParticles':5, 
                'ListDecayParticles3B':6, 'DefaultInputValues':7, 
                'AddTreeLevelUnitarityLimits':8, 'RenConditionsDecays':9}   
        }
        super().__init__(*args,**kwargs)
    
    def update_SPheno(self):
        parameters=self.updated_modelparameters
        sci=extract_Lagrangian_terms(parameters,
                                     Lagrangian_dimension=4,
                                     Lorentz_structures=1).get('Symbol')

        sciIN=get_input_parameters_IN(sci,suffix='IN')

        self.SPheno['Properties'
                   ]['BoundaryLowScaleInput']=get_BoundaryLowScaleInput(sci,sciIN)

        self.SPheno['Properties'
                   ]['MINPAR']=get_MINPAR(sciIN)

        self.SPheno['Properties'
                   ]['ParametersToSolveTadpoles'
                    ]=get_tadpoles_and_bilinears(parameters,exclude=[])[0]

        self.SPheno['Properties'][
            'DEFINITION']['MatchingConditions']=get_SM_MatchingConditions(parameters)

        DP,DP3B=get_decay_particles(DecayParticles   = ['Fu', 'Fe', 'Fd', 'hh'],
                            DecayParticles3B = ['Fu', 'Fe', 'Fd'])
        self.SPheno['Properties'
                   ]['ListDecayParticles']=DP
        self.SPheno['Properties'
                   ]['ListDecayParticles3B']=DP3B

        self.SPheno['Properties'][
            'DefaultInputValues'].update( 
                             get_sm_DefaultInputValues(parameters,sci,sciIN) )
        return self.SPheno
    
    def update_SARAH_full(self):
        kk=self.parse_model_parameters()
        kk=self.update_SPheno()

In [16]:
us=update_SARAH(model='SSDM')
kk=us.update_SARAH_full()

In [17]:
pd.DataFrame(us.SPheno)

,Index,Properties
AddTreeLevelUnitarityLimits,8,True
BoundaryLowScaleInput,3,"[[Lambda1/2, Lambda1IN]]"
DEFINITION,4,"{'MatchingConditions': [['v', 'vSM'], ['Yd', 'YdSM'], ['Ye', 'YeSM'], ['Yu', 'YuSM'], ['g1', 'g1SM'], ['g2', 'g2SM'], ['g3', 'g3SM']]}"
DefaultInputValues,7,{'Lambda1IN': 0.27}
ListDecayParticles,5,"[Fu, Fe, Fd, hh]"
ListDecayParticles3B,6,"[[Fu, ""Fu.f90""], [Fe, ""Fe.f90""], [Fd, ""Fd.f90""]]"
MINPAR,1,"[[1, Lambda1IN]]"
OnlyLowEnergySPheno,0,True
ParametersToSolveTadpoles,2,[mu2]
RenConditionsDecays,9,"[[dCosTW, 1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))], [dSinTW, -dCosTW/Tan[ThetaW]], [dg2, 1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiV..."


# Output

In [18]:
class SARAH_output(update_SARAH):
    def __init__(self,*args, **kwargs):
        self.output_dir=kwargs.get('output_dir')
        super().__init__(*args,**kwargs)

    def check_output_dir(self):
        if not self.output_dir:
            self.output_dir='{}{}'.format(
                                self.model_dir,
                                self.model_name
                                        ).strip()

        if not re.search('\/$',self.output_dir):
            self.output_dir='{}/'.format(self.output_dir)

        if not os.path.exists(self.output_dir):
            os.mkdir( self.output_dir )

    def to_math(self,particles_name ='particles.m',
                     parameters_name='parameters.m',
                     SPheno_name    ='SPheno.m'):
        kk=self.update_SARAH_full()
        kk=self.check_output_dir()
        # Prepare output files
        dirpath='{}{}'.format(self.output_dir,self.model_name)
        if not re.search('\/$',dirpath):
            dirpath='{}/'.format(dirpath)
        if not os.path.isdir(dirpath):
            os.mkdir(dirpath)
        model_path     ='{}{}'.format(dirpath,self.model_file_name)
        particles_path ='{}{}'.format(dirpath,particles_name)
        parameters_path='{}{}'.format(dirpath,parameters_name)
        SPheno_path    ='{}{}'.format(dirpath,SPheno_name)

        #write output files
        WRITE=True
        if os.path.isfile(model_path):
            CHKW=input('File {} exists.Overwrite? (y/n)'.format(
                                  model_path ))
            if re.search('^\s*[nN][oOtT]{0,2}',CHKW):
                WRITE=False
        if WRITE:
            f=open(model_path,'w')
            f.write(self.model_file)
            f.close()
            
        PDF=to_defintions(self.updated_modelparticles,symbol='Field')
        dfnp=pd.DataFrame(PDF)
        kk=to_math(dfnp,particles_path)
        PDF=to_defintions(self.updated_modelparameters,symbol='Symbol')
        dfnp=pd.DataFrame(PDF)
        kk=to_math(dfnp,parameters_path,definitions='ParameterDefinitions')
        SP=pd.DataFrame(self.SPheno)
        kk=to_SPheno(SP,SPheno_path,dictentries=['DefaultInputValues'])        

In [19]:
us=SARAH_output(model='SSDM',output_dir='tmp')
kk=us.to_math()

In [20]:
df=pd.DataFrame(us.updated_modelparticles).dropna(subset=['Description']).reset_index(drop=True)
df[df['Description'].str.contains('Ghost')]

,Block,Definition,Description,Field,Name,Parents,Properties,rotation,update_Description
7,None,GaugeES,B-Boson Ghost,gB,B-Boson Ghost,VB,"{'Lorentz': 'Scalar', 'Group': ' U[1]'}",None,NaN
8,None,GaugeES,W-Boson Ghost,gWB,W-Boson Ghost,VWB,"{'Lorentz': 'Scalar', 'Group': ' SU[2]'}",None,NaN
9,None,EWSB,Gluon Ghost,gG,Gluon Ghost,VG,"{'Lorentz': 'Scalar', 'Group': ' SU[3]'}",None,NaN
13,GaugeSector,EWSB,Photon Ghost,gP,NaN,NaN,{'Lorentz': 'Ghost'},NaN,NaN
14,GaugeSector,EWSB,Z-Boson Ghost,gZ,NaN,NaN,{'Lorentz': 'Ghost'},NaN,NaN
15,GaugeSector,EWSB,Positive W+ - Boson Ghost,gWp,NaN,NaN,{'Lorentz': 'Ghost'},NaN,NaN
16,GaugeSector,EWSB,Negative W+ - Boson Ghost,gWpC,NaN,NaN,{'Lorentz': 'Ghost'},NaN,NaN


In [21]:
us=SARAH_output(model='SSDM',output_dir='tmp')
kk=us.to_math()

File tmp/SSDM/SSDM.m exists.Overwrite? (y/n)y
